In [369]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Rucurrent Neural Network for forecasting bitcoin price 

Created on Sun Jun 14 2022

@author: chenqian
"""

# univariate multi-step vector-output stacked lstm example
import csv
import pickle
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
import numpy as np
import pandas as pd
# multi-step data preparation
from numpy import array


In [532]:
dfv = pd.read_csv('Full_data.csv')        
df_n = pd.read_csv('WHO-COVID-19-global-data.csv') 

In [476]:
df_n.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187180 entries, 0 to 187179
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Date_reported      187180 non-null  object
 1   Country_code       186391 non-null  object
 2   Country            187180 non-null  object
 3   WHO_region         187180 non-null  object
 4   New_cases          187180 non-null  int64 
 5   Cumulative_cases   187180 non-null  int64 
 6   New_deaths         187180 non-null  int64 
 7   Cumulative_deaths  187180 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 11.4+ MB


In [534]:
# ETL process(country Canada, United States of America, Spain, 2021-01-018 to 2022-02-27)
df = df_n[df_n['Country'].isin(['Spain','Canada','United States of America'])]
df = df[df['New_cases'].notnull()]
df = df[df['Cumulative_cases'].notnull()]
df = df[df['New_deaths'].notnull()]
df = df[df['Cumulative_deaths'].notnull()]
df = df.reset_index(drop = True)
df_c = df[(df['Country'] == 'Canada') & (df['Date_reported'] >= '2021-01-18') & (df['Date_reported'] <= '2022-02-27')]
df_s = df[(df['Country'] == 'Spain') & (df['Date_reported'] >= '2021-01-18') & (df['Date_reported'] <= '2022-02-27')]
df_u = df[(df['Country_code'] == 'US') & (df['Date_reported'] >= '2021-01-18') & (df['Date_reported'] <= '2022-02-27')]
df_c = df_c.reset_index(drop = True)
df_s = df_s.reset_index(drop = True)
df_u = df_u.reset_index(drop = True)


df_c['week'] = pd.DataFrame([j for j in range(int(df_c.shape[0]/7)) for i in range(7)])
df_s['week'] = pd.DataFrame([j for j in range(int(df_c.shape[0]/7)) for i in range(7)])
df_u['week'] = pd.DataFrame([j for j in range(int(df_c.shape[0]/7)) for i in range(7)])


df_c['transmission_rate'] = df_c['New_cases']/df_c['Cumulative_cases']*10
df_s['transmission_rate'] = df_s['New_cases']/df_c['Cumulative_cases']*10
df_u['transmission_rate'] = df_u['New_cases']/df_c['Cumulative_cases']

df_c['case_fatality_rate'] = df_c['Cumulative_deaths']/df_c['Cumulative_cases']*10
df_s['case_fatality_rate'] = df_s['Cumulative_deaths']/df_c['Cumulative_cases']*10
df_u['case_fatality_rate'] = df_u['Cumulative_deaths']/df_c['Cumulative_cases']
df_c = df_c.groupby('week').mean().reset_index()
df_s = df_s.groupby('week').mean().reset_index()
df_u = df_u.groupby('week').mean().reset_index()
df_c['Country'] = 'Canada'
df_s['Country'] = 'Spain'
df_u['Country'] = 'United States'

# useful vaccinated data
dfv = dfv[dfv['location'].isin(['Canada','United States', 'Spain'])][['location', 'date','people_fully_vaccinated']]

dfv_c = dfv[(dfv['location'] == 'Canada') & (dfv['date'] >= '2021-01-18') & (dfv['date'] <= '2022-02-27')]
dfv_u = dfv[(dfv['location'] == 'United States') & (dfv['date'] >= '2021-01-18') & (dfv['date'] <= '2022-02-27')]
dfv_s = dfv[(dfv['location'] == 'Spain') & (dfv['date'] >= '2021-01-18') & (dfv['date'] <= '2022-02-27')]
dfv_c = dfv_c.reset_index(drop = True)
dfv_s = dfv_s.reset_index(drop = True)
dfv_u = dfv_u.reset_index(drop = True)
dfv_c['week'] = pd.DataFrame([j for j in range(int(dfv_c.shape[0]/7)) for i in range(7)])
dfv_u['week'] = pd.DataFrame([j for j in range(int(dfv_u.shape[0]/7)) for i in range(7)])
dfv_s['week'] = pd.DataFrame([j for j in range(int(dfv_s.shape[0]/7)) for i in range(7)])
dfv_c = dfv_c.groupby('week').max().reset_index()
dfv_u = dfv_u.groupby('week').max().reset_index()
dfv_s = dfv_s.groupby('week').max().reset_index()

dfv_c['people_fully_vaccinated'] = dfv_c['people_fully_vaccinated']/sum(dfv_c['people_fully_vaccinated']**2)**(1/2)
dfv_s['people_fully_vaccinated'] = dfv_s['people_fully_vaccinated']/sum(dfv_s['people_fully_vaccinated']**2)**(1/2)
dfv_u['people_fully_vaccinated'] = dfv_u['people_fully_vaccinated']/sum(dfv_u['people_fully_vaccinated']**2)**(1/2)

df_c['people_fully_vaccinated'] = dfv_c['people_fully_vaccinated']*10
df_s['people_fully_vaccinated'] = dfv_s['people_fully_vaccinated']*10
df_u['people_fully_vaccinated'] = dfv_u['people_fully_vaccinated']

df_auxa = pd.concat([df_u,df_s])
df_all = pd.concat([df_auxa,df_c])
df_all


,week,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,transmission_rate,case_fatality_rate,Country,people_fully_vaccinated
0,0,180639.857143,2.426124e+07,3206.428571,419182.142857,0.251007,0.582209,United States,0.003383
1,1,152664.285714,2.540351e+07,3172.000000,441359.714286,0.201686,0.583122,United States,0.005900
2,2,122647.285714,2.636346e+07,2873.714286,462480.000000,0.155989,0.588041,United States,0.009556
3,3,95619.714286,2.710961e+07,2832.857143,481534.285714,0.117941,0.593875,United States,0.014120
4,4,67457.571429,2.766952e+07,2075.428571,499964.714286,0.081149,0.601341,United States,0.018450
5,5,66546.857143,2.812456e+07,1926.714286,513348.428571,0.078047,0.602363,United States,0.023990
6,6,59997.571429,2.857008e+07,1480.000000,525208.142857,0.068733,0.601796,United States,0.029319
7,7,54818.714286,2.896339e+07,1213.857143,534523.714286,0.061314,0.598237,United States,0.035409
8,8,54420.857143,2.934460e+07,894.714286,541855.714286,0.059356,0.591433,United States,0.040800
9,9,59734.142857,2.974209e+07,873.428571,547928.285714,0.063282,0.581174,United States,0.047673


In [529]:
df_c

,week,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,transmission_rate,case_fatality_rate
0,0,5957.142857,7.199967e+05,157.000000,18311.000000,0.082810,0.254316
1,1,4769.428571,7.568670e+05,139.000000,19386.714286,0.063055,0.256138
2,2,3851.857143,7.864524e+05,115.428571,20257.142857,0.048995,0.257572
3,3,3221.428571,8.108121e+05,79.000000,20923.857143,0.039741,0.258061
4,4,2897.142857,8.314149e+05,59.142857,21391.142857,0.034821,0.257289
5,5,2983.714286,8.522220e+05,48.428571,21768.000000,0.034999,0.255432
6,6,2898.428571,8.727414e+05,39.571429,22066.285714,0.033199,0.252844
7,7,3067.428571,8.935150e+05,30.285714,22306.000000,0.034310,0.249651
8,8,3405.142857,9.162041e+05,30.428571,22524.571429,0.037140,0.245855
9,9,4226.571429,9.428467e+05,29.857143,22735.000000,0.044775,0.241146


In [515]:
def count_endpoint(pre):
    aux = 0
    for i in y_predend:
        if i > 0:
            aux = aux + 1
        else:
            break
    return aux         

def predict(model, x_Input,cfr_input, pfv_input, N_steps_in, N_features, num_predi):
    result = []
    for i in range(num_predi):
        data = array(x_Input)
        datau = np.append(data,  array(cfr_input))
        datau = np.append(datau, array(pfv_input))
        datau = datau.reshape((1, N_steps_in, N_features))
        predicted = model.predict(datau, verbose=0)
        result.append(predicted[0][0])
        x_Input = x_Input[1:]
        x_Input.append(predicted[0][0])
        datau = []
    return result

def predict_end_u(model, x_Input, N_steps_in, N_features, num_predi):
    result = []
    for i in range(num_predi):
        data = array(x_Input)
        datau = np.append(data,  array([[0.287071,0.287071]]))
        datau = np.append(datau, array([[0.173468,0.173468]]))
        datau = datau.reshape((1, N_steps_in, N_features))
        predicted = model.predict(datau, verbose=0)
        result.append(predicted[0][0])
        x_Input = x_Input[1:]
        x_Input.append(predicted[0][0])
        datau = []
    return result

def predict_end_s(model, x_Input, N_steps_in, N_features, num_predi):
    result = []
    for i in range(num_predi):
        data = array(x_Input)
        datau = np.append(data,  array([[0.30535,0.30535]]))
        datau = np.append(datau, array([[1.85592,1.85592]]))
        datau = datau.reshape((1, N_steps_in, N_features))
        predicted = model.predict(datau, verbose=0)
        result.append(predicted[0][0])
        x_Input = x_Input[1:]
        x_Input.append(predicted[0][0])
        datau = []
    return result

def predict_end_c(model, x_Input, N_steps_in, N_features, num_predi):
    result = []
    for i in range(num_predi):
        data = array(x_Input)
        datau = np.append(data,  array([[0.111194,0.111194]]))
        datau = np.append(datau, array([[1.92078,1.92078]]))
        datau = datau.reshape((1, N_steps_in, N_features))
        predicted = model.predict(datau, verbose=0)
        result.append(predicted[0][0])
        x_Input = x_Input[1:]
        x_Input.append(predicted[0][0])
        datau = []
    return result


def plotResults(yhat, y, num_obser):
    raw_seq_x = range(1, num_obser + 1)
    raw_seq_y = y[0:num_obser]
    raw_seq_pre = yhat[0:num_obser]
    plt.plot(raw_seq_x, raw_seq_y)
    plt.plot(raw_seq_x, raw_seq_pre)
    plt.show()
    
    
# split a univariate sequence into samples
def split_sequence(sequence,n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):# find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
        
    return array(X), array(y)

# split a univariate sequence into train data and test data
def train_test_sep(sequence, n):
    train_data, test_data = list(), list()
    train_data = sequence.iloc[0:(len(sequence)-n)]
    test_data  = sequence.iloc[(len(sequence) -n):len(sequence)]
    return train_data, test_data

# Root Mean Squared Error 
def root_mean_squared_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return (np.mean((y_true - y_pred)**2))**(1/2)





In [516]:
aux_rmse_u,aux_endp_u = [], []
for i in range(30):   
    y_pred = [10] 
    while (y_pred[0] - float(test_set['transmission_rate'])) > 0.02: 
        train_set, test_set = train_test_sep(df_u, 1)
        # choose a number of time steps
        n_steps_in, n_steps_out = 2, 1

        # split into samples
        train_tr, test_tr = split_sequence(train_set['transmission_rate'],n_steps_in, n_steps_out)
        train_cfr, test_cfr = split_sequence(train_set['case_fatality_rate'],n_steps_in, n_steps_out)
        train_pfv, test_pfv = split_sequence(train_set['people_fully_vaccinated'],n_steps_in, n_steps_out)
        train_aux = np.append(train_tr, train_cfr, axis=1)
        X = np.append(train_aux, train_pfv, axis=1)
        y = test_tr

        # summarize the data
        # for i in range(len(X)):
        #print(X[i], y[i])


        # reshape from [samples, timesteps] into [samples, timesteps, features]
        n_features = 1
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # define model
        model = Sequential()
        model.add(Bidirectional(LSTM(200, activation='relu'), input_shape=(n_steps_in*3, n_features)))
        model.add(Dropout(0.2))
        model.add(Dense(n_steps_out))
        model.compile(optimizer='adam', loss='mse')

        # weights = model.layers[0].get_weights()[0]
        # biases = model.layers[0].get_weights()[1]
        # fit model
        model.fit(X, y, epochs=200, verbose=0, validation_split=0.05)
        # prediction for RMSE
        t_set = list(train_set['transmission_rate'])
        cfr = train_set['case_fatality_rate'][(len(t_set) -n_steps_in):len(t_set)]
        pfv = train_set['people_fully_vaccinated'][(len(t_set) -n_steps_in):len(t_set)]
        x_input = t_set[(len(t_set) -n_steps_in):len(t_set)]
        y_pred = predict(model, x_input,cfr,pfv, n_steps_in*3, n_features, 1)
    # prediction for Endpoints
    t_set = list(df_u['transmission_rate'])
    x_input = t_set[(len(t_set) -n_steps_in):len(t_set)]
    y_predend = predict_end_u(model, x_input, n_steps_in*3, n_features, 48)
    print(i, y_pred)
    aux_endp_u.append(count_endpoint(y_predend))
    aux_rmse_u.append(y_pred)



0 [0.032337695]
1 [0.03081454]
2 [0.02157205]
3 [0.027362105]
4 [0.023897406]
5 [0.020806279]
6 [0.037731867]
7 [0.020094749]
8 [0.032258958]
9 [0.033241436]
10 [0.03433644]
11 [0.037648432]
12 [0.03093787]
13 [0.023436021]
14 [0.024711676]
15 [0.03559944]
16 [0.02501027]
17 [0.03266945]
18 [0.033506412]
19 [0.028001]
20 [0.023816511]
21 [0.031387854]
22 [0.029201679]
23 [0.020264199]
24 [0.030909447]
25 [0.02874998]
26 [0.029144377]
27 [0.033995185]
28 [0.02288996]
29 [0.034727346]


In [525]:
aux_rmse_s,aux_endp_s = [], []
for i in range(30):   
    y_pred = [10] 
    while (y_pred[0] - float(test_set['transmission_rate'])) > 0.02: 
        train_set, test_set = train_test_sep(df_s, 1)
        # choose a number of time steps
        n_steps_in, n_steps_out = 2, 1

        # split into samples
        train_tr, test_tr = split_sequence(train_set['transmission_rate'],n_steps_in, n_steps_out)
        train_cfr, test_cfr = split_sequence(train_set['case_fatality_rate'],n_steps_in, n_steps_out)
        train_pfv, test_pfv = split_sequence(train_set['people_fully_vaccinated'],n_steps_in, n_steps_out)
        train_aux = np.append(train_tr, train_cfr, axis=1)
        X = np.append(train_aux, train_pfv, axis=1)
        y = test_tr

        # summarize the data
        # for i in range(len(X)):
        #print(X[i], y[i])


        # reshape from [samples, timesteps] into [samples, timesteps, features]
        n_features = 1
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # define model
        model = Sequential()
        model.add(Bidirectional(LSTM(200, activation='relu'), input_shape=(n_steps_in*3, n_features)))
        model.add(Dropout(0.1))
        model.add(Dense(n_steps_out))
        model.compile(optimizer='adam', loss='mse')

        # weights = model.layers[0].get_weights()[0]
        # biases = model.layers[0].get_weights()[1]
        # fit model
        model.fit(X, y, epochs=200, verbose=0, validation_split=0.05)
        # prediction for RMSE
        t_set = list(train_set['transmission_rate'])
        cfr = train_set['case_fatality_rate'][(len(t_set) -n_steps_in):len(t_set)]
        pfv = train_set['people_fully_vaccinated'][(len(t_set) -n_steps_in):len(t_set)]
        x_input = t_set[(len(t_set) -n_steps_in):len(t_set)]
        y_pred = predict(model, x_input,cfr,pfv, n_steps_in*3, n_features, 1)
    # prediction for Endpoints
    t_set = list(df_u['transmission_rate'])
    x_input = t_set[(len(t_set) -n_steps_in):len(t_set)]
    y_predend = predict_end_s(model, x_input, n_steps_in*3, n_features, 48)
    print(i, y_pred)
    aux_endp_s.append(count_endpoint(y_predend))
    aux_rmse_s.append(y_pred)



0 [-0.095750816]
1 [-0.021900455]
2 [-0.06175368]
3 [-0.013729322]
4 [-0.084764145]
5 [0.031735156]
6 [-0.1736387]
7 [-0.106636204]
8 [-0.083959706]
9 [-0.08511771]
10 [-0.12642664]
11 [-0.13314156]
12 [0.03948596]
13 [-0.12350329]
14 [-0.032537982]
15 [-0.019373812]
16 [-0.08380233]
17 [-0.026332092]
18 [-0.053437453]
19 [-0.21884684]
20 [0.024607506]
21 [-0.16044204]
22 [-0.11381674]
23 [-0.08613138]
24 [-0.06904781]
25 [-0.1460911]
26 [-0.107512675]
27 [-0.019993993]
28 [-0.14951208]
29 [-0.15112957]


In [518]:
aux_rmse_c,aux_endp_c = [], []
for i in range(30):   
    y_pred = [10] 
    while (abs(y_pred[0] - float(test_set['transmission_rate']))) > 0.02: 
        train_set, test_set = train_test_sep(df_c, 1)
        # choose a number of time steps
        n_steps_in, n_steps_out = 2, 1

        # split into samples
        train_tr, test_tr = split_sequence(train_set['transmission_rate'],n_steps_in, n_steps_out)
        train_cfr, test_cfr = split_sequence(train_set['case_fatality_rate'],n_steps_in, n_steps_out)
        train_pfv, test_pfv = split_sequence(train_set['people_fully_vaccinated'],n_steps_in, n_steps_out)
        train_aux = np.append(train_tr, train_cfr, axis=1)
        X = np.append(train_aux, train_pfv, axis=1)
        y = test_tr

        # summarize the data
        # for i in range(len(X)):
        #print(X[i], y[i])


        # reshape from [samples, timesteps] into [samples, timesteps, features]
        n_features = 1
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # define model
        model = Sequential()
        model.add(Bidirectional(LSTM(200, activation='relu'), input_shape=(n_steps_in*3, n_features)))
        model.add(Dropout(0.2))
        model.add(Dense(n_steps_out))
        model.compile(optimizer='adam', loss='mse')

        # weights = model.layers[0].get_weights()[0]
        # biases = model.layers[0].get_weights()[1]
        # fit model
        model.fit(X, y, epochs=200, verbose=0, validation_split=0.05)
        # prediction for RMSE
        t_set = list(train_set['transmission_rate'])
        cfr = train_set['case_fatality_rate'][(len(t_set) -n_steps_in):len(t_set)]
        pfv = train_set['people_fully_vaccinated'][(len(t_set) -n_steps_in):len(t_set)]
        x_input = t_set[(len(t_set) -n_steps_in):len(t_set)]
        y_pred = predict(model, x_input,cfr,pfv, n_steps_in*3, n_features, 1)
    # prediction for Endpoints
    t_set = list(df_u['transmission_rate'])
    x_input = t_set[(len(t_set) -n_steps_in):len(t_set)]
    y_predend = predict_end_c(model, x_input, n_steps_in*3, n_features, 48)
    print(i, y_pred)
    aux_endp_c.append(count_endpoint(y_predend))
    aux_rmse_c.append(y_pred)



0 [0.009345552]
1 [-0.014808377]
2 [0.020758]
3 [0.005204548]
4 [0.013974222]
5 [0.010391047]
6 [-0.028889254]
7 [0.009238129]
8 [0.0132199265]
9 [0.018172076]
10 [0.0034167795]
11 [0.027780775]
12 [0.0012284177]
13 [0.021373596]
14 [0.013043263]
15 [-0.005963585]
16 [-0.04205038]
17 [-0.010537095]
18 [0.006997372]
19 [0.011426881]
20 [0.012798096]
21 [-0.07700123]
22 [-0.0005653361]
23 [-0.0030380418]
24 [-0.04113666]
25 [0.0034043386]
26 [0.013293182]
27 [-0.015252497]
28 [-0.03369633]
29 [0.012936283]


In [540]:
fore_s = {'RMSE':aux_rmse_s,
        'End Date':aux_endp_s,
        'Country':['Spain' for i in range(len(aux_rmse_s))]
           }
fore_c = {'RMSE':aux_rmse_c,
        'End Date':aux_endp_c,
        'Country':['Canada' for i in range(len(aux_rmse_s))]
           }
fore_u = {'RMSE':aux_rmse_u,
        'End Date':aux_endp_u,
        'Country':['US' for i in range(len(aux_rmse_s))]
           }
f_s = pd.DataFrame(fore_s)
f_c = pd.DataFrame(fore_c)
f_u = pd.DataFrame(fore_u)
df_a = pd.concat([f_u, f_c])
df_p = pd.concat([df_a, f_s]) 
df_p['RMSE'] = [abs(i[0]) for i in df_p['RMSE']] 

In [598]:
df_p.to_csv('result_data.csv',index=False)